In [1]:
import numpy as np

In [48]:
class Board:
    def __init__(self, horizontal_size = 3, vertical_size = 3):
        self.horizontal_size = horizontal_size
        self.vertical_size = vertical_size
        self.board = np.zeros((horizontal_size * vertical_size), dtype=int)
    
    def make_move(self, move:int,  player):
        """
        Makes a move on the board for a player at the given position
        :param move: the position to make the move at
        :param player: the player making the move (1 or -1)
        """
        if self.board[move] == 0:
            self.board[move] = player
        else:
            raise ValueError("Invalid move: a player has already made a move at this position")
        
    def check_winner(self):
        """
        Checks if a player has won the game
        :return: the player that has won the game, 0 if no player has won
        """
        # Check for horizontal win
        for i in range(self.horizontal_size):
            if all(self.board[i*self.vertical_size:(i+1)*self.vertical_size] == 1):
                return 1
            elif all(self.board[i*self.vertical_size:(i+1)*self.vertical_size] == -1):
                return -1
        
        # Check for vertical win
        for i in range(self.vertical_size):
            if all(self.board[i::self.vertical_size] == 1):
                return 1
            elif all(self.board[i::self.vertical_size] == -1):
                return -1
        # Check for diagonal win
        if all(self.board[0::self.vertical_size+1] == 1) or all(self.board[self.vertical_size-1:-1:self.vertical_size-1] == 1):
            return 1
        elif all(self.board[0::self.vertical_size+1] == -1) or all(self.board[self.vertical_size-1:-1:self.vertical_size-1] == -1):
            return -1
        
        return 0
    
    def check_draw(self):
        """
        Checks if the game is a draw
        :return: True if the game is a draw, False otherwise
        """
        return (0 not in set([i for i in self.board])) and (self.check_winner() == 0)
    
    def get_possible_moves(self):
        """
        Gets all possible moves on the board
        :return: a list of possible moves
        """
        return np.where(self.board == 0)
        
    def get_board(self):
        return self.board[i*self.horizontal_size:(i+1)*self.horizontal_size]

    def print_board(self):
        for i in range(self.vertical_size):
            print(self.board[i*self.horizontal_size:(i+1)*self.horizontal_size])
            

In [55]:
board = Board()
print(board)
board.print_board()

[0 0 0]
[0 0 0]
[0 0 0]


In [56]:

class TicTacToe:
    def __init__(self):
        self.board = Board()
        self.current_player = 1
    
    def make_move(self, move:int):
        """
        Makes a move on the board for the current player and starts the turn to the other player
        :param move: the position to make the move at
        """
        self.board.make_move(move, self.current_player)
        self.current_player *= -1
    
    def check_winner(self):
        """
        Checks if a player has won the game
        :return: the player that has won the game, 0 if no player has won
        """
        return self.board.check_winner()
    
    def check_draw(self):
        """
        Checks if the game is a draw
        :return: True if the game is a draw, False otherwise
        """
        return self.board.check_draw()
    
    def get_possible_moves(self):
        """
        Gets all possible moves on the board
        :return: a list of possible moves
        """
        return self.board.get_possible_moves()
    
    def get_board(self):
        return self.board.get_board()
    
    def get_current_player(self):
        return self.current_player

In [63]:
class Agent:
    def __init__(self):
        pass
    
    def get_move(self, game):
        pass

class RandomPlayer(Agent):
    def __init__(self):
        pass
    
    def get_move(self, game):
        return np.random.choice(game.get_possible_moves()[0])
    
    def play_one_game(self, game):
        """Agent plays one game of Tic Tac Toe against itself and returns the winner
        :param game: the game to play
        :return: the winner of the game
        """
        while True:
            if game.check_winner() != 0:
                return game.check_winner()
            if game.check_draw():
                return 0
            game.make_move(self.get_move(game))

In [61]:
game = TicTacToe()
player = RandomPlayer()
while game.check_winner() == 0 and not game.check_draw():
    move = player.get_move(game)
    game.make_move(move)
    game.board.print_board()
    print("current player: ", game.get_current_player())
    print(game.check_winner())
    print(game.check_draw())


[0 0 0]
[0 0 0]
[0 1 0]
current player:  -1
0
False
[0 0 0]
[ 0 -1  0]
[0 1 0]
current player:  1
0
False
[0 0 0]
[ 0 -1  0]
[1 1 0]
current player:  -1
0
False
[0 0 0]
[ 0 -1  0]
[ 1  1 -1]
current player:  1
0
False
[0 1 0]
[ 0 -1  0]
[ 1  1 -1]
current player:  -1
0
False
[0 1 0]
[-1 -1  0]
[ 1  1 -1]
current player:  1
0
False
[1 1 0]
[-1 -1  0]
[ 1  1 -1]
current player:  -1
0
False
[1 1 0]
[-1 -1 -1]
[ 1  1 -1]
current player:  1
-1
False


In [ ]:
class Node:
    def __init__(self, game, parent=None):
        self.game = game
        self.parent = parent
        self.children = []
        self.visits = 0
        self.wins = 0
    
    def add_child(self, child):
        self.children.append(child)
    
    def update(self, result):
        self.visits += 1
        self.wins += result
    
    def get_ucb(self):
        return float('inf') if self.visits==0 else self.wins/self.visits + np.sqrt(2*np.log(self.parent.visits)/self.visits)
    
    def get_best_child(self):
        return max(self.children, key=lambda x: x.get_ucb())
    
    def add_all_children(self):
        for move in self.game.get_possible_moves()[0]:
            new_game = copy.deepcopy(self.game)
            new_game.make_move(move)
            self.add_child(Node(new_game, self))
    


In [ ]:
class MCTS:
    def __init__(self, game: TicTacToe, agent: Agent, num_simulations=1000):
        self.game = game
        self.num_simulations = num_simulations
        self.agent = agent
        self.root = Node(game)
    
    def search_leaf(self, node):
        if len(node.children) == 0:
            return node
        else:
            move = node.get_best_child()
            self.game.make_move(move)
            return self.search_leaf(node.get_best_child())
    
    def rollout(self, node):
        """Simulates a game from the current node to the end and returns the winner
        :param node: the node to simulate from
        """
        winner = self.player.play_one_game(node.game)
        return winner
        
    
    def backpropagete(self, node, result):
        while node != None:
            node.update(result)
            node = node.parent
    
    def do_one_step(self, node): # here a deepcopy of the game is suggested by copilot
        game = copy.deepcopy(node.game)
        node = self.search_leaf(node)
        if node.visits == 0:
            return game.check_winner()
        else:
            if game.check_winner() != 0:
                return node.game.check_winner()
            elif game.check_draw():
                return 0
            else:
                new_game = copy.deepcopy(node.game)
                new_game.make_move(self.agent.get_move(new_game))
                return self.simulate(Node(new_game))

